In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
import math
import re
import requests
import random

In [3]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier

In [4]:

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay,plot_confusion_matrix
from sklearn.model_selection import GridSearchCV
import itertools

In [5]:
df_train = pd.read_csv(r'C:\Users\saurabh.vaidya\CROSS-SELL\NON_CIBIL\Train.csv',sep=',',encoding='utf-8',parse_dates=True)
df_test = pd.read_csv(r'C:\Users\saurabh.vaidya\CROSS-SELL\NON_CIBIL\Test.csv',sep=',',encoding='utf-8',parse_dates=True)
# df_validation = pd.read_csv(r'C:\Users\saurabh.vaidya\37TH_NAP_NEW\Validation_with_cibil_v2.csv',sep=',',encoding='utf-8',parse_dates=True)

In [6]:
print("Shape of train:", df_train.shape)
print("Shape of test:", df_test.shape)
# print("Shape of validation", df_validation.shape)

Shape of train: (15846, 41)
Shape of test: (6791, 41)


In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15846 entries, 0 to 15845
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   RESIDENTIAL_STATUS           15846 non-null  object 
 1   INCOME_SLAB                  15846 non-null  float64
 2   NUMBER_OF_DEPENDENT          15846 non-null  float64
 3   HFL_CHANNEL                  15846 non-null  object 
 4   HFL_STATUS                   15846 non-null  object 
 5   LOAN_SECTOR_TYPE             15846 non-null  object 
 6   LOAN_REPAYMENT_MODE          15846 non-null  object 
 7   SUB_SCHEME                   15846 non-null  object 
 8   PRODUCT_DESC                 15846 non-null  object 
 9   LOAN_STATUS                  15846 non-null  object 
 10  INT_TYPE                     15846 non-null  object 
 11  INSTALLMENT_TYPE             15846 non-null  object 
 12  ROI                          15846 non-null  float64
 13  LOAN_TENURE     

In [8]:
df_train['ABC_ASSOCIATION_NEW'] = df_train['ABC_ASSOCIATION'] - df_train['MF_TARGET']
df_test['ABC_ASSOCIATION_NEW'] = df_test['ABC_ASSOCIATION'] - df_test['MF_TARGET']

### IV

In [9]:
def iv_woe(data, target, bins, show_woe=False):
    
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    
    #Extract Column Names
    cols = data.columns
    
    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target])]:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})
        d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
        d.columns = ['Cutoff', 'N', 'Events']
        d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()
        d['Non-Events'] = d['N'] - d['Events']
        d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()
        d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])
        d['IV'] = d['WoE'] * (d['% of Events'] - d['% of Non-Events'])
        d.insert(loc=0, column='Variable', value=ivars)
        print("Information value of " + ivars + " is " + str(round(d['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)
        woeDF=pd.concat([woeDF,d], axis=0)

        #Show WOE Table
        if show_woe == True:
            print(d)
    return newDF, woeDF

In [10]:
# iv, woe = iv_woe(data = df_train, target = 'MF_TARGET', bins = 4, show_woe = True)

In [11]:
# iv.to_excel('IV.xlsx', index = False)

### VIF

In [12]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(df1):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = df1.columns
    vif["VIF"] = [float(format(variance_inflation_factor(df1.values, i),'f') )for i in range(df1.shape[1])]

    return(vif)

In [13]:
df2 = df_train._get_numeric_data()

In [14]:
# df2 = df2.drop(columns = 'INCOME')

In [15]:
# VIF=calc_vif(df2)

In [16]:
# VIF.sort_values('VIF',ascending=False)

In [17]:
# VIF.to_csv('VIF.csv', index = False)

In [18]:
# cols2 = VIF[VIF['VIF'] < 15]['variables'].to_list()

# df3 = df2[cols2].copy()
# VIF2=calc_vif(df3)

In [19]:
# VIF2.to_csv('VIF2.csv', index = False)

In [20]:
# VIF2.sort_values('VIF',ascending=False)

In [21]:
# cols3 = VIF2[VIF2['VIF'] < 5]['variables'].to_list()

# df4 = df3[cols3].copy()
# VIF3=calc_vif(df4)

In [22]:
# VIF3.sort_values('VIF',ascending=False)

### Chi-Square Test

In [23]:
# catg = df_train.select_dtypes(include=['object'])

In [24]:
# from scipy.stats import chi2_contingency
# #Chi-square test of independence.
# p_value = []

# for col in catg.columns:
#     contigency= pd.crosstab('TARGET_37', catg[col])
#     c, p, dof, expected = chi2_contingency(contigency, correction=False)
    
#     print(col)
#     if p > 0.05: 
#         print("p_value: ",round(p,3))
#         h0 = "Accept H0"
#         print(h0)
#     else:
#         print("p_value: ",round(p,3))
#         h0 = "Reject H0"
#         print(h0)
#     print()
    
#     p_value.append(
#         {
#             'VARIABLE': col,
#             'P-VALUE': p,
#             'H0': h0
#         }
#     )

In [25]:
# p_value1 = pd.DataFrame(p_value)

In [26]:
# p_value1.to_csv('p_value1.csv', index = False)

### Information Gain

In [27]:
def calc_entropy(column):
    """
    Calculate entropy given a series, list, or numpy array.
    """
    # Compute the counts of each given value in the column
    counts = np.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            # use log from math and set base to 2
            entropy += prob * math.log(prob, 2)
    
    return -entropy

In [28]:
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on and target.
    """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    # Find the unique values in the column
    values = data[split_name].unique()
    
    # Make two subsets of the data, based on the unique values
#     print(data[data[split_name] == values[0]])
#     print()
#     print(data[data[split_name] == values[1]])

    print(split_name, ':', values)
    left_split = (data[data[split_name] == values[0]])
    right_split = (data[data[split_name] == values[1]])
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0])
        to_subtract += prob * calc_entropy(subset[target_name])
        
    # Return information gain
    return original_entropy - to_subtract

In [29]:
# inf_gain = []

# for i in df_train.columns:
#     try:
#         inf = calc_information_gain(df_train, i, 'MF_TARGET')
#     except:
#         inf = "error"
        
#     inf_gain.append({
#         'VARIABLE': i,
#         'Information Gain': inf
#     })

# df_inf_gain = pd.DataFrame(inf_gain)
# df_inf_gain.to_excel('IG.xlsx', index = False)
# df_inf_gain.head()

In [30]:
model_var = [
    'RESIDENTIAL_STATUS', #CATG
    'HFL_CHANNEL', #CATG
    'HFL_STATUS', #CATG
    'LOAN_REPAYMENT_MODE', #CATG
    'PRODUCT_DESC', #CATG
    'LOAN_TENURE', #NUM
    'PORTFOLIO_DESC', #CATG
    'DAYS_TILL_FIRST_DISBURSAL', #NUM
    'TOTAL_DISBURSED_AMOUNT', #NUM
    'EDU_DETAIL', #CATG
    'TIER', #CATG
    'CUSTOMER_CATEGORY', #CATG
    'CUSTOMER_TYPE', #CATG
]

In [31]:
# import scorecardpy as sc
# appended_data = []
# for col in model_var:
#     Max = df_train[[col, 'TARGET_37']]
#     Max_Del = sc.woebin(Max, y = 'TARGET_37')
#     new = Max_Del[col]
#     # store DataFrame in list
#     appended_data.append(new)

# appended_data = pd.concat(appended_data)

# appended_data.to_excel('WOE_BIN.xlsx', index = False)

### WOE Binning

In [32]:
num_biv = pd.read_excel(r'C:\Users\saurabh.vaidya\CROSS-SELL\NON_CIBIL\model_v1\bivariate\numerical_bi_variate.xlsx')

In [33]:
num_biv.head()

,variable,bin,good,bad,count,percentage
0,RESIDENTIAL_STATUS,"NRI%,%RI%,%PIO",12073,1162,13235,0.088
1,RESIDENTIAL_STATUS,RESIDENT,2255,356,2611,0.136
2,INCOME_SLAB,"[-inf,13000.0)",775,123,898,0.137
3,INCOME_SLAB,"[13000.0,27000.0)",2162,261,2423,0.108
4,INCOME_SLAB,"[27000.0,35000.0)",7515,639,8154,0.078


In [34]:
def find(text):
    str1 = text

    return (str1.translate({ord(i): None for i in '%'}))

In [35]:
num_biv['bin'] = num_biv.apply(lambda x: find(x['bin']), axis=1)

### NUMERICAL VARIABLE BINNING

In [36]:
def num_label(row, col, conditions):
    if len(conditions) == 3:
        if (row[col] < conditions[0]):
            return 0
        elif (conditions[0] <= row[col]) & (row[col] < conditions[1]):
            return 1
        elif (conditions[1] <= row[col]) & (row[col] < conditions[2]):
            return 2
        else:
            return 3
        
    elif len(conditions) == 4:
        if (row[col] < conditions[0]):
            return 0
        elif (conditions[0] <= row[col]) & (row[col] < conditions[1]):
            return 1
        elif (conditions[1] <= row[col]) & (row[col] < conditions[2]):
            return 2
        elif (conditions[2] <= row[col]) & (row[col] < conditions[3]):
            return 3
        else:
            return 4
    
    elif len(conditions) == 5:
        if (row[col] < conditions[0]):
            return 0
        elif (conditions[0] <= row[col]) & (row[col] < conditions[1]):
            return 1
        elif (conditions[1] <= row[col]) & (row[col] < conditions[2]):
            return 2
        elif (conditions[2] <= row[col]) & (row[col] < conditions[3]):
            return 3
        elif (conditions[3] <= row[col]) & (row[col] < conditions[4]):
            return 4
        else:
            return 5
    
    else:
        return 'NONE'
        
def catg_bin(row, col, num_biv):
    num_biv_1 = num_biv[num_biv['variable'] == col].reset_index(drop = True)
    conditions = []
    for i in num_biv_1.index:
        conditions.append(float(num_biv_1['bin'][i][1:-1].split(',')[1]))
    conditions = conditions[:-1]
    print(conditions)
    row[col + str('_BIN')] = row.apply(lambda row: num_label(row, col, conditions), axis=1)
    return row

In [37]:
df_train = catg_bin(df_train, 'LOAN_TENURE', num_biv)
df_test = catg_bin(df_test, 'LOAN_TENURE', num_biv)
# df_validation = catg_bin(df_validation, 'LOAN_TENURE', num_biv)

[50.0, 200.0, 290.0, 340.0]
[50.0, 200.0, 290.0, 340.0]


In [38]:
df_train['LOAN_TENURE_BIN'].value_counts()

2    8253
1    3513
4    2286
3     905
0     889
Name: LOAN_TENURE_BIN, dtype: int64

In [39]:
df_train['LOAN_TENURE_BIN'].mask(df_train['LOAN_TENURE_BIN'] == 3, 2, inplace = True)
df_test['LOAN_TENURE_BIN'].mask(df_test['LOAN_TENURE_BIN'] == 3, 2, inplace = True)
# df_validation['LOAN_TENURE_BIN'].mask(df_validation['LOAN_TENURE_BIN'] == 3, 2, inplace = True)


In [40]:
df_train['LOAN_TENURE_BIN'].value_counts()

2    9158
1    3513
4    2286
0     889
Name: LOAN_TENURE_BIN, dtype: int64

In [41]:
df_train = catg_bin(df_train, 'DAYS_TILL_FIRST_DISBURSAL', num_biv)
df_test = catg_bin(df_test, 'DAYS_TILL_FIRST_DISBURSAL', num_biv)
# df_validation = catg_bin(df_validation, 'DAYS_TILL_FIRST_DISBURSAL', num_biv)

[12.0, 14.0, 18.0]
[12.0, 14.0, 18.0]


In [42]:
df_train['DAYS_TILL_FIRST_DISBURSAL_BIN'].value_counts()

0    5541
3    5042
1    4286
2     977
Name: DAYS_TILL_FIRST_DISBURSAL_BIN, dtype: int64

In [43]:
df_train['DAYS_TILL_FIRST_DISBURSAL_BIN'].mask(df_train['DAYS_TILL_FIRST_DISBURSAL_BIN'] == 0, 1, inplace = True)
df_test['DAYS_TILL_FIRST_DISBURSAL_BIN'].mask(df_test['DAYS_TILL_FIRST_DISBURSAL_BIN'] == 0, 1, inplace = True)
# df_validation['DAYS_TILL_FIRST_DISBURSAL_BIN'].mask(df_validation['DAYS_TILL_FIRST_DISBURSAL_BIN'] == 0, 1, inplace = True)


In [44]:
df_train['DAYS_TILL_FIRST_DISBURSAL_BIN'].value_counts()

1    9827
3    5042
2     977
Name: DAYS_TILL_FIRST_DISBURSAL_BIN, dtype: int64

In [45]:
df_train = catg_bin(df_train, 'TOTAL_DISBURSED_AMOUNT', num_biv)
df_test = catg_bin(df_test, 'TOTAL_DISBURSED_AMOUNT', num_biv)
# df_validation = catg_bin(df_validation, 'TOTAL_DISBURSED_AMOUNT', num_biv)

[1700000.0, 1900000.0, 2100000.0, 3800000.0, 5600000.0]
[1700000.0, 1900000.0, 2100000.0, 3800000.0, 5600000.0]


In [46]:
df_train['TOTAL_DISBURSED_AMOUNT_BIN'].value_counts()

0    6135
3    3308
1    2900
5    1516
4    1058
2     929
Name: TOTAL_DISBURSED_AMOUNT_BIN, dtype: int64

In [47]:
df_train['TOTAL_DISBURSED_AMOUNT_BIN'].mask(df_train['TOTAL_DISBURSED_AMOUNT_BIN'] == 0, 1, inplace = True)
df_test['TOTAL_DISBURSED_AMOUNT_BIN'].mask(df_test['TOTAL_DISBURSED_AMOUNT_BIN'] == 0, 1, inplace = True)
# df_validation['TOTAL_DISBURSED_AMOUNT_BIN'].mask(df_validation['TOTAL_DISBURSED_AMOUNT_BIN'] == 0, 1, inplace = True)


In [48]:
df_train['TOTAL_DISBURSED_AMOUNT_BIN'].value_counts()

1    9035
3    3308
5    1516
4    1058
2     929
Name: TOTAL_DISBURSED_AMOUNT_BIN, dtype: int64

### CATEGORICAL VARIABLE BINNING 

In [49]:
def label(row, col, conditions):
    for i in range(len(conditions)):
        if row[col] in conditions[i]:
            return 'Group' + str(i)
        
    
def catg_bin(row, col, num_biv):
    num_biv_1 = num_biv[num_biv['variable'] == col].reset_index(drop = True)
    conditions = []
    for i in num_biv_1.index:
        conditions.append(num_biv_1['bin'][i].split(','))
    
    print(conditions)
    row[col + str('_BIN')] = row.apply(lambda row: label(row, col, conditions), axis=1)
    return row

In [50]:
df_train = catg_bin(df_train, 'SCHEME', num_biv)
df_test = catg_bin(df_test, 'SCHEME', num_biv)
# df_validation = catg_bin(df_validation, 'SCHEME', num_biv)

[['UC BT', 'INDUSTRIAL', 'LAP RESI INSURANCE', 'LOAN AGAINST PLOT COMMERCIAL', 'U C BT AND EXTENSION', 'LOAN AGAINST PROPERTY INFORMAL PLOT RESIDENTIAL', 'LOAN AGAINST PROPERTY RESIDENTIAL', 'LAP COMM INSURANCE', 'EXTENSION', 'REFINANCE', 'COMM VACANT', 'COMM LET OUT', 'REPAIR AND RENOVATION', 'BT RENOVATION', 'RESI VACANT', 'CONSTRUCTION LOAN', 'LAND AND PROJECT CONSTRUCTION LOAN', 'BT', 'INSURANCE', 'PLOT CONSTRUCTION', 'BUILDER PURCHASE'], ['SELF CONSTRUCTION', 'PLOT AND HOME CONSTRUCTION LOAN', 'HOME CONSTRUCTION LOAN', 'RESALE PURCHASE', 'HOME IMPROVEMENT LOAN', 'BT AND TOP UP', 'HOME EXTENSION LOAN', 'PLOT  HOME CONSTRUCTION LOAN'], ['TOP UP', 'HL FIXED FOR 5 YRS', 'HOME LOAN PURCHASE', 'LOAN AGAINST PLOT RESIDENTIAL', 'PLOT PURCHASE LOAN', 'PLOT PURCHASE', 'SOCP'], ['NEW', 'RESIDENTIAL', 'COMMERCIAL', 'RESI LET OUT', 'PROJECT CONSTRUCTION COST']]
[['UC BT', 'INDUSTRIAL', 'LAP RESI INSURANCE', 'LOAN AGAINST PLOT COMMERCIAL', 'U C BT AND EXTENSION', 'LOAN AGAINST PROPERTY INFORMAL

In [51]:
df_train['SCHEME_BIN'].value_counts()

Group2    10408
Group3     2447
Group1     2107
Group0      884
Name: SCHEME_BIN, dtype: int64

In [52]:
df_train = catg_bin(df_train, 'PRODUCT_DESC', num_biv)
df_test = catg_bin(df_test, 'PRODUCT_DESC', num_biv)
# df_validation = catg_bin(df_validation, 'PRODUCT_DESC', num_biv)

[['HOME LOAN_PRIME RETAIL', 'MICRO CONSTRUCTION FINANCE', 'MICRO LAP_AFFORDABLE', 'HOME LOAN_AHF'], ['HOME LOAN'], ['LOAN AGAINST PROPERTY'], ['LAP_AHF', 'HOME LOAN_INFORMAL', 'COMMERCIAL PURCHASE (CP)', 'COMMERCIAL PROPERTY PURCHASE_AHF', 'CONSTRUCTION FINANCE', 'LOAN AGAINST PROPERTY_INFORMAL', 'LEASE RENTAL DISCOUNTING', 'CONSTRUCTION FINANCE INTERNAL']]
[['HOME LOAN_PRIME RETAIL', 'MICRO CONSTRUCTION FINANCE', 'MICRO LAP_AFFORDABLE', 'HOME LOAN_AHF'], ['HOME LOAN'], ['LOAN AGAINST PROPERTY'], ['LAP_AHF', 'HOME LOAN_INFORMAL', 'COMMERCIAL PURCHASE (CP)', 'COMMERCIAL PROPERTY PURCHASE_AHF', 'CONSTRUCTION FINANCE', 'LOAN AGAINST PROPERTY_INFORMAL', 'LEASE RENTAL DISCOUNTING', 'CONSTRUCTION FINANCE INTERNAL']]


In [53]:
df_train['PRODUCT_DESC_BIN'].value_counts()

Group0    10295
Group1     2662
Group3     1998
Group2      891
Name: PRODUCT_DESC_BIN, dtype: int64

In [54]:
df_train = catg_bin(df_train, 'RESIDENTIAL_STATUS', num_biv)
df_test = catg_bin(df_test, 'RESIDENTIAL_STATUS', num_biv)
# df_validation = catg_bin(df_validation, 'RESIDENTIAL_STATUS', num_biv)

[['NRI', 'RI', 'PIO'], ['RESIDENT']]
[['NRI', 'RI', 'PIO'], ['RESIDENT']]


In [55]:
df_train['RESIDENTIAL_STATUS_BIN'].value_counts()

Group0    13235
Group1     2611
Name: RESIDENTIAL_STATUS_BIN, dtype: int64

In [56]:
df_train['RESIDENTIAL_STATUS_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
df_test['RESIDENTIAL_STATUS_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
# df_validation['RESIDENTIAL_STATUS_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)


In [57]:
df_train['RESIDENTIAL_STATUS_BIN'].value_counts()

0    13235
1     2611
Name: RESIDENTIAL_STATUS_BIN, dtype: int64

In [58]:
df_train = catg_bin(df_train, 'HFL_CHANNEL', num_biv)
df_test = catg_bin(df_test, 'HFL_CHANNEL', num_biv)
# df_validation = catg_bin(df_validation, 'HFL_CHANNEL', num_biv)

[['REFERRAL PARTNER', 'DIRECT'], ['(DSA)DIRECT SELLING AGENT']]
[['REFERRAL PARTNER', 'DIRECT'], ['(DSA)DIRECT SELLING AGENT']]


In [59]:
df_train['HFL_CHANNEL_BIN'].value_counts()

Group0    12608
Group1     3238
Name: HFL_CHANNEL_BIN, dtype: int64

In [60]:
df_train['HFL_CHANNEL_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
df_test['HFL_CHANNEL_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
# df_validation['HFL_CHANNEL_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)


In [61]:
df_train['HFL_CHANNEL_BIN'].value_counts()

0    12608
1     3238
Name: HFL_CHANNEL_BIN, dtype: int64

In [62]:
df_train = catg_bin(df_train, 'HFL_STATUS', num_biv)
df_test = catg_bin(df_test, 'HFL_STATUS', num_biv)
# df_validation = catg_bin(df_validation, 'HFL_STATUS', num_biv)

[['ACTIVE'], ['INACTIVE']]
[['ACTIVE'], ['INACTIVE']]


In [63]:
df_train['HFL_STATUS_BIN'].value_counts()

Group0    12847
Group1     2999
Name: HFL_STATUS_BIN, dtype: int64

In [64]:
df_train['HFL_STATUS_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
df_test['HFL_STATUS_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
# df_validation['HFL_STATUS_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)


In [65]:
df_train['HFL_STATUS_BIN'].value_counts()

0    12847
1     2999
Name: HFL_STATUS_BIN, dtype: int64

In [66]:
df_train = catg_bin(df_train, 'LOAN_REPAYMENT_MODE', num_biv)
df_test = catg_bin(df_test, 'LOAN_REPAYMENT_MODE', num_biv)
# df_validation = catg_bin(df_validation, 'LOAN_REPAYMENT_MODE', num_biv)

[['PDC', 'ACH'], ['ECS']]
[['PDC', 'ACH'], ['ECS']]


In [67]:
df_train['LOAN_REPAYMENT_MODE_BIN'].value_counts()

Group0    14935
Group1      911
Name: LOAN_REPAYMENT_MODE_BIN, dtype: int64

In [68]:
df_train['LOAN_REPAYMENT_MODE_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
df_test['LOAN_REPAYMENT_MODE_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
# df_validation['LOAN_REPAYMENT_MODE_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)


In [69]:
df_train['LOAN_REPAYMENT_MODE_BIN'].value_counts()

0    14935
1      911
Name: LOAN_REPAYMENT_MODE_BIN, dtype: int64

In [70]:
df_train = catg_bin(df_train, 'PORTFOLIO_DESC', num_biv)
df_test = catg_bin(df_test, 'PORTFOLIO_DESC', num_biv)
# df_validation = catg_bin(df_validation, 'PORTFOLIO_DESC', num_biv)

[['MICRO CF', 'AFFORDABLE'], ['PRIME', 'INFRML', 'CF']]
[['MICRO CF', 'AFFORDABLE'], ['PRIME', 'INFRML', 'CF']]


In [71]:
df_train['PORTFOLIO_DESC_BIN'].value_counts()

Group0    11602
Group1     4244
Name: PORTFOLIO_DESC_BIN, dtype: int64

In [72]:
df_train['PORTFOLIO_DESC_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
df_test['PORTFOLIO_DESC_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
# df_validation['PORTFOLIO_DESC_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)


In [73]:
df_train['PORTFOLIO_DESC_BIN'].value_counts()

0    11602
1     4244
Name: PORTFOLIO_DESC_BIN, dtype: int64

In [74]:
df_train = catg_bin(df_train, 'EDU_DETAIL', num_biv)
df_test = catg_bin(df_test, 'EDU_DETAIL', num_biv)
# df_validation = catg_bin(df_validation, 'EDU_DETAIL', num_biv)

[['DOC', 'UP TO 9TH', 'HG_SCHOOL', 'MAT', '12TH', 'O', 'OTH', 'DPLM'], ['UG'], ['GRAD'], ['PG', 'PRO']]
[['DOC', 'UP TO 9TH', 'HG_SCHOOL', 'MAT', '12TH', 'O', 'OTH', 'DPLM'], ['UG'], ['GRAD'], ['PG', 'PRO']]


In [75]:
df_train['EDU_DETAIL_BIN'].value_counts()

Group2    7849
Group1    3558
Group0    2424
Group3    2015
Name: EDU_DETAIL_BIN, dtype: int64

In [76]:
df_train['EDU_DETAIL_BIN'].replace({'Group0': 0, 'Group1': 1, 'Group2': 2, 'Group3': 3}, inplace = True)
df_test['EDU_DETAIL_BIN'].replace({'Group0': 0, 'Group1': 1, 'Group2': 2, 'Group3': 3}, inplace = True)
# df_validation['EDU_DETAIL_BIN'].replace({'Group0': 0, 'Group1': 1, 'Group2': 2, 'Group3': 3}, inplace = True)


In [77]:
df_train['EDU_DETAIL_BIN'].value_counts()

2    7849
1    3558
0    2424
3    2015
Name: EDU_DETAIL_BIN, dtype: int64

In [78]:
df_train = catg_bin(df_train, 'TIER', num_biv)
df_test = catg_bin(df_test, 'TIER', num_biv)
# df_validation = catg_bin(df_validation, 'TIER', num_biv)

[['TIER3'], ['TIER2'], ['TIER1']]
[['TIER3'], ['TIER2'], ['TIER1']]


In [79]:
df_train['TIER_BIN'].value_counts()

Group1    7739
Group2    5376
Group0    2731
Name: TIER_BIN, dtype: int64

In [80]:
df_train['TIER_BIN'].replace({'Group2': 0, 'Group1': 1, 'Group0': 2}, inplace = True)
df_test['TIER_BIN'].replace({'Group2': 0, 'Group1': 1, 'Group0': 2}, inplace = True)
# # df_validation['TIER_BIN'].replace({'Group2': 0, 'Group1': 1, 'Group0': 2}, inplace = True)


In [81]:
df_train['TIER_BIN'].value_counts()

1    7739
0    5376
2    2731
Name: TIER_BIN, dtype: int64

In [82]:
df_train = catg_bin(df_train, 'CUSTOMER_CATEGORY', num_biv)
df_test = catg_bin(df_test, 'CUSTOMER_CATEGORY', num_biv)
# df_validation = catg_bin(df_validation, 'CUSTOMER_CATEGORY', num_biv)

[['LOW'], ['HIGH', 'MEDIUM']]
[['LOW'], ['HIGH', 'MEDIUM']]


In [83]:
df_train['CUSTOMER_CATEGORY_BIN'].value_counts()

Group0    8689
Group1    7157
Name: CUSTOMER_CATEGORY_BIN, dtype: int64

In [84]:
df_train['CUSTOMER_CATEGORY_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
df_test['CUSTOMER_CATEGORY_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
# df_validation['CUSTOMER_CATEGORY_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)


In [85]:
df_train['CUSTOMER_CATEGORY_BIN'].value_counts()

0    8689
1    7157
Name: CUSTOMER_CATEGORY_BIN, dtype: int64

In [86]:
df_train = catg_bin(df_train, 'CUSTOMER_TYPE', num_biv)
df_test = catg_bin(df_test, 'CUSTOMER_TYPE', num_biv)
# df_validation = catg_bin(df_validation, 'CUSTOMER_TYPE', num_biv)

[['I'], ['C', 'INDIVIDUAL', 'CORPORATE']]
[['I'], ['C', 'INDIVIDUAL', 'CORPORATE']]


In [87]:
df_train['CUSTOMER_TYPE_BIN'].value_counts()

Group1    11688
Group0     4158
Name: CUSTOMER_TYPE_BIN, dtype: int64

In [88]:
df_train['CUSTOMER_TYPE_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
df_test['CUSTOMER_TYPE_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
# df_validation['CUSTOMER_TYPE_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)


In [89]:
df_train['CUSTOMER_CATEGORY_BIN'].value_counts()

0    8689
1    7157
Name: CUSTOMER_CATEGORY_BIN, dtype: int64

In [90]:
from sklearn.preprocessing import MinMaxScaler

df_train['LOAN_TENURE'] = MinMaxScaler().fit_transform(np.array(df_train['LOAN_TENURE']).reshape(-1,1)) 
df_test['LOAN_TENURE'] = MinMaxScaler().fit_transform(np.array(df_test['LOAN_TENURE']).reshape(-1,1)) 


In [91]:
df_train['DAYS_TILL_FIRST_DISBURSAL'] = MinMaxScaler().fit_transform(np.array(df_train['DAYS_TILL_FIRST_DISBURSAL']).reshape(-1,1)) 
df_test['DAYS_TILL_FIRST_DISBURSAL'] = MinMaxScaler().fit_transform(np.array(df_test['DAYS_TILL_FIRST_DISBURSAL']).reshape(-1,1)) 


In [92]:
df_train['TOTAL_DISBURSED_AMOUNT'] = MinMaxScaler().fit_transform(np.array(df_train['TOTAL_DISBURSED_AMOUNT']).reshape(-1,1)) 
df_test['TOTAL_DISBURSED_AMOUNT'] = MinMaxScaler().fit_transform(np.array(df_test['TOTAL_DISBURSED_AMOUNT']).reshape(-1,1)) 


In [93]:
train = df_train.copy()
test = df_test.copy()
# validation = df_validation.copy()

### Model Development

In [107]:
df_train = train.copy()
df_test = test.copy()
# df_validation = validation.copy()

In [108]:
model_var = [
    'RESIDENTIAL_STATUS_BIN', #CATG
    'HFL_CHANNEL_BIN', #CATG
    'HFL_STATUS_BIN', #CATG
#     'LOAN_REPAYMENT_MODE_BIN', #CATG
    'PRODUCT_DESC_BIN', #CATG
    'SCHEME_BIN', #CATG
#     'LOAN_TENURE_BIN', #NUM
    'LOAN_TENURE', #NUM
    'PORTFOLIO_DESC_BIN', #CATG
#     'DAYS_TILL_FIRST_DISBURSAL_BIN', #NUM
#     'TOTAL_DISBURSED_AMOUNT_BIN', #NUM
#     'DAYS_TILL_FIRST_DISBURSAL', #NUM
    'TOTAL_DISBURSED_AMOUNT', #NUM
    'EDU_DETAIL_BIN', #CATG
    'TIER_BIN', #CATG
    'CUSTOMER_CATEGORY_BIN', #CATG
    'CUSTOMER_TYPE_BIN', #CATG
    'PRINCIPAL_OUTSTANDING',
    'ABC_ASSOCIATION_NEW'
]

In [109]:
df_train1 = df_train[model_var]

cat_vars = [i for i in df_train1.columns if df_train1[i].dtype == 'object']

for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(df_train1[var], prefix=var)
    df_train1=df_train1.join(cat_list)
    
df_train1=df_train1.drop(cat_vars,axis=1)

df_test1 = df_test[model_var]
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(df_test1[var], prefix=var)
    df_test1=df_test1.join(cat_list)
    
df_test1=df_test1.drop(cat_vars,axis=1)

# df_validation1 = df_validation[model_var]
# for var in cat_vars:
#     cat_list='var'+'_'+var
#     cat_list = pd.get_dummies(df_validation1[var], prefix=var)
#     df_validation1=df_validation1.join(cat_list)
    
# df_validation1=df_validation1.drop(cat_vars,axis=1)


print("Training set : ",df_train1.shape)
print("Test set : ",df_test1.shape)
# print("Validation set : ",df_validation1.shape)

Training set :  (15846, 20)
Test set :  (6791, 20)


In [110]:
X_train = df_train1.copy()
y_train = df_train['MF_TARGET']
print("Shape of X_train: ", X_train.shape)
print("Shape of y_train: ", y_train.shape)
print("\n")

X_test = df_test1.copy()
y_test = df_test['MF_TARGET']
print("Shape of X_test: ", X_test.shape)
print("Shape of y_test: ", y_test.shape)
print("\n")

# X_validation = df_validation1.copy()
# y_validation = df_validation['MF_TARGET']
# print("Shape of X_validation: ", X_validation.shape)
# print("Shape of y_validation: ", y_validation.shape)
# print("\n")


Shape of X_train:  (15846, 20)
Shape of y_train:  (15846,)


Shape of X_test:  (6791, 20)
Shape of y_test:  (6791,)




In [111]:
X_train.columns

Index(['RESIDENTIAL_STATUS_BIN', 'HFL_CHANNEL_BIN', 'HFL_STATUS_BIN',
       'LOAN_TENURE', 'PORTFOLIO_DESC_BIN', 'TOTAL_DISBURSED_AMOUNT',
       'EDU_DETAIL_BIN', 'TIER_BIN', 'CUSTOMER_CATEGORY_BIN',
       'CUSTOMER_TYPE_BIN', 'PRINCIPAL_OUTSTANDING', 'ABC_ASSOCIATION_NEW',
       'PRODUCT_DESC_BIN_Group0', 'PRODUCT_DESC_BIN_Group1',
       'PRODUCT_DESC_BIN_Group2', 'PRODUCT_DESC_BIN_Group3',
       'SCHEME_BIN_Group0', 'SCHEME_BIN_Group1', 'SCHEME_BIN_Group2',
       'SCHEME_BIN_Group3'],
      dtype='object')

In [112]:
f_imp = [
#         'PRODUCT_DESC_BIN_Group3', 'SCHEME_BIN_Group0',
    'PORTFOLIO_DESC_BIN','SCHEME_BIN_Group2', 'SCHEME_BIN_Group0',
       'ABC_ASSOCIATION_NEW', 'SCHEME_BIN_Group1',
       'TOTAL_DISBURSED_AMOUNT', 'PRINCIPAL_OUTSTANDING',
       'PRODUCT_DESC_BIN_Group2', 'RESIDENTIAL_STATUS_BIN', 
       'PRODUCT_DESC_BIN_Group1', 'PRODUCT_DESC_BIN_Group3',
        'EDU_DETAIL_BIN',  'TIER_BIN', #CATG
#        'EDU_DETAIL_BIN_Group0', 'EDU_DETAIL_BIN_Group2',
#        'EDU_DETAIL_BIN_Group1', 'EDU_DETAIL_BIN_Group3',
#         'HFL_CHANNEL_BIN', 'CUSTOMER_CATEGORY_BIN',
#     'LOAN_TENURE',         'DAYS_TILL_FIRST_DISBURSAL',
]

In [113]:
X_train = X_train[f_imp]
X_test = X_test[f_imp]

In [114]:
# X_train.to_csv('X_train.csv', index = False)
# y_train.to_csv('y_train.csv', index = False)

In [115]:
# X_test.to_csv('X_test.csv', index = False)
# y_test.to_csv('y_test.csv', index = False)

In [116]:
X_train.head()

,PORTFOLIO_DESC_BIN,SCHEME_BIN_Group2,SCHEME_BIN_Group0,ABC_ASSOCIATION_NEW,SCHEME_BIN_Group1,TOTAL_DISBURSED_AMOUNT,PRINCIPAL_OUTSTANDING,PRODUCT_DESC_BIN_Group2,RESIDENTIAL_STATUS_BIN,PRODUCT_DESC_BIN_Group1,PRODUCT_DESC_BIN_Group3,EDU_DETAIL_BIN,TIER_BIN
0,1,1,0,0,0,0.001339,0.00,0,0,1,0,2,2
1,0,1,0,0,0,0.000557,2086398.32,0,0,0,0,0,1
2,0,1,0,0,0,0.003447,0.00,0,0,0,0,2,1
3,0,0,0,0,1,0.000934,3967620.00,0,0,0,0,0,2
4,0,1,0,0,0,0.000269,1181949.44,0,0,0,0,1,1


In [117]:
X_train.columns

Index(['PORTFOLIO_DESC_BIN', 'SCHEME_BIN_Group2', 'SCHEME_BIN_Group0',
       'ABC_ASSOCIATION_NEW', 'SCHEME_BIN_Group1', 'TOTAL_DISBURSED_AMOUNT',
       'PRINCIPAL_OUTSTANDING', 'PRODUCT_DESC_BIN_Group2',
       'RESIDENTIAL_STATUS_BIN', 'PRODUCT_DESC_BIN_Group1',
       'PRODUCT_DESC_BIN_Group3', 'EDU_DETAIL_BIN', 'TIER_BIN'],
      dtype='object')

### XG Boost

In [141]:
import xgboost as xgb
xgb_cl = xgb.XGBClassifier(objective='binary:logistic', booster='gbtree',learning_rate=0.21,max_depth=8,
                               base_score=0.4, n_estimators=200, num_parallel_tree=3, eval_metric='auc',
                               tree_method='hist', grow_policy='lossguide', scale_pos_weight = 9,
                               gamma = 89, colsample_bytree = 0.5
                              ).fit(X_train, y_train)

y_pred_xg1 = xgb_cl.predict(X_train)
y_pred_xg = xgb_cl.predict(X_test)

X_train_prob = list(xgb_cl.predict_proba(X_train)[:,1])
X_test_prob = list(xgb_cl.predict_proba(X_test)[:,1])

In [142]:
# print()
print("ROC AUC Score for train:", roc_auc_score(y_train, X_train_prob) * 100)
print("ROC AUC Score for test:", roc_auc_score(y_test, X_test_prob) * 100)
# print("ROC AUC Score for valid:", roc_auc_score(y_validation, y_pred_xg_v) * 100)

ROC AUC Score for train: 72.45030368869674
ROC AUC Score for test: 70.80266117838512


In [143]:
feat_imp = pd.DataFrame.from_dict(zip(list(X_train.columns), xgb_cl.feature_importances_*100)).sort_values(by = 1, ascending = False)
# feat_imp.to_excel('feature_imp_xgb.xlsx', index = False)
feat_imp

,0,1
10,PRODUCT_DESC_BIN_Group3,27.060205
0,PORTFOLIO_DESC_BIN,15.233268
8,RESIDENTIAL_STATUS_BIN,9.881929
3,ABC_ASSOCIATION_NEW,9.562695
9,PRODUCT_DESC_BIN_Group1,8.017418
2,SCHEME_BIN_Group0,7.042231
4,SCHEME_BIN_Group1,6.988342
6,PRINCIPAL_OUTSTANDING,6.353462
11,EDU_DETAIL_BIN,4.207214
5,TOTAL_DISBURSED_AMOUNT,4.078832


In [144]:
feat_imp[0].values

array(['PRODUCT_DESC_BIN_Group3', 'PORTFOLIO_DESC_BIN',
       'RESIDENTIAL_STATUS_BIN', 'ABC_ASSOCIATION_NEW',
       'PRODUCT_DESC_BIN_Group1', 'SCHEME_BIN_Group0',
       'SCHEME_BIN_Group1', 'PRINCIPAL_OUTSTANDING', 'EDU_DETAIL_BIN',
       'TOTAL_DISBURSED_AMOUNT', 'TIER_BIN', 'SCHEME_BIN_Group2',
       'PRODUCT_DESC_BIN_Group2'], dtype=object)

In [145]:
def ks(data=None,target=None, prob=None):
    data['target_xgb'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 10)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['total'] = (grouped.sum()[target] + grouped.sum()['target_xgb'])
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target_xgb']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2%}'.format)
    kstable['nonevent_rate'] = (kstable.nonevents / data['target_xgb'].sum()).apply('{0:.2%}'.format)
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target_xgb'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2%}'.format)
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2%}'.format)
    
    kstable['ks_stats'] = np.round(((kstable['events'] / kstable['events'].sum()).cumsum() -(kstable['nonevents'] / kstable['nonevents'].sum()).cumsum()), 4) * 100
    kstable['max_ks'] = kstable['ks_stats'].apply(lambda x: '*****' if x == kstable['ks_stats'].max() else '')
    
    kstable.index = range(1,11)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 20)
#    print(kstable)
#    print(type(kstable))    
    #Display KS
    from colorama import Fore
#   print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
#     print(type(kstable))
    return(kstable)


In [146]:
train['Prob_score_XGB'] = X_train_prob
test['Prob_score_XGB'] = X_test_prob
    
xg_train_ks = ks(train,'MF_TARGET','Prob_score_XGB')
xg_test_ks = ks(test,'MF_TARGET','Prob_score_XGB')

In [147]:
# xg_train_ks.to_csv('xgboost_ks_train.csv', index = False)
xg_train_ks

,min_prob,max_prob,total,events,nonevents,event_rate,nonevent_rate,cum_eventrate,cum_noneventrate,KS,ks_stats,max_ks
Decile,,,,,,,,,,,,
1,0.628764,0.872401,1585,452,1133,29.78%,7.91%,29.78%,7.91%,21.9,21.87,
2,0.565062,0.628150,1583,273,1310,17.98%,9.14%,47.76%,17.05%,30.7,30.71,
3,0.516227,0.564908,1583,176,1407,11.59%,9.82%,59.35%,26.87%,32.5,32.48,*****
4,0.480091,0.516056,1587,150,1437,9.88%,10.03%,69.24%,36.90%,32.3,32.34,
5,0.421330,0.480061,1585,120,1465,7.91%,10.22%,77.14%,47.12%,30.0,30.02,
6,0.391361,0.421328,1570,100,1470,6.59%,10.26%,83.73%,57.38%,26.3,26.34,
7,0.355292,0.391335,1594,86,1508,5.67%,10.52%,89.39%,67.91%,21.5,21.48,
8,0.297931,0.355260,1588,59,1529,3.89%,10.67%,93.28%,78.58%,14.7,14.70,
9,0.268650,0.297481,1463,68,1395,4.48%,9.74%,97.76%,88.32%,9.4,9.44,


In [148]:
# xg_test_ks.to_csv('xgboost_ks_test.csv', index = False)
xg_test_ks

,min_prob,max_prob,total,events,nonevents,event_rate,nonevent_rate,cum_eventrate,cum_noneventrate,KS,ks_stats,max_ks
Decile,,,,,,,,,,,,
1,0.637510,0.872401,677,184,493,28.48%,8.02%,28.48%,8.02%,20.5,20.46,
2,0.579719,0.636535,673,108,565,16.72%,9.19%,45.20%,17.22%,28.0,27.98,
3,0.535220,0.579691,686,87,599,13.47%,9.75%,58.67%,26.97%,31.7,31.70,*****
4,0.487035,0.535131,679,51,628,7.89%,10.22%,66.56%,37.18%,29.4,29.38,
5,0.438244,0.486824,645,55,590,8.51%,9.60%,75.08%,46.79%,28.3,28.29,
6,0.402793,0.437983,708,47,661,7.28%,10.76%,82.35%,57.54%,24.8,24.81,
7,0.365836,0.402556,683,38,645,5.88%,10.50%,88.24%,68.04%,20.2,20.20,
8,0.321299,0.365711,681,27,654,4.18%,10.64%,92.41%,78.68%,13.7,13.73,
9,0.288214,0.321004,625,32,593,4.95%,9.65%,97.37%,88.33%,9.0,9.04,


In [149]:


# Init classifier
def best_result(eta, md, npt, gam, train, test):
    xgb_cl = xgb.XGBClassifier(objective='binary:logistic', booster='gbtree',learning_rate=eta,max_depth=md,
                               base_score=0.4, n_estimators=200, num_parallel_tree=npt, eval_metric='auc',
                               tree_method='hist', grow_policy='lossguide', scale_pos_weight = 9,
                               gamma = gam, colsample_bytree = 0.5
                              ).fit(X_train, y_train)

    y_pred_xg1 = xgb_cl.predict(X_train)
    y_pred_xg = xgb_cl.predict(X_test)
    
    X_train_prob = list(xgb_cl.predict_proba(X_train)[:,1])
    X_test_prob = list(xgb_cl.predict_proba(X_test)[:,1])
    print("max depth: " , md , ", num parallel tree: " , npt , "gamma: " , gam)
    train_res = roc_auc_score(y_train, X_train_prob) * 100
    test_res = roc_auc_score(y_test, X_test_prob) * 100
    
    train['Prob_score_XGB'] = X_train_prob
    test['Prob_score_XGB'] = X_test_prob

    xg_train_ks = ks(train,'MF_TARGET','Prob_score_XGB')
    xg_test_ks = ks(test,'MF_TARGET','Prob_score_XGB')
    
    train_3rd_cap = xg_train_ks.reset_index()['cum_eventrate'][2]
    test_3rd_cap = xg_test_ks.reset_index()['cum_eventrate'][2]
    
#     print("ROC AUC Score for train:", train_res)
#     print("ROC AUC Score for test:", test_res)
    return eta, md, npt, gam, train_res, test_res, train_3rd_cap, test_3rd_cap

In [150]:
my_dict = {"LearningRate": [], "Max_Depth":[], "Num_Parallel_Tree":[], "Gamma":[],
           "Train_AUC": [], "Test_AUC": [], "Train_3rd_DC": [], "Test_3rd_DC": []};
for i in [0.21, 0.215, 0.219]:
    for j in [5, 7, 6]:
        for k in [2, 4, 5]:
            for l in [87, 88, 90, 91]:
                eta, md, npt, gam, train_res, test_res, train_3rd_cap, test_3rd_cap = best_result(i, j, k, l, train, test)
                my_dict["LearningRate"].append(eta)
                my_dict["Max_Depth"].append(md)
                my_dict["Num_Parallel_Tree"].append(npt)
                my_dict["Gamma"].append(gam)
                my_dict["Train_AUC"].append(train_res)
                my_dict["Test_AUC"].append(test_res)
                my_dict["Train_3rd_DC"].append(train_3rd_cap)
                my_dict["Test_3rd_DC"].append(test_3rd_cap)


                print("------------------------------------------------------------")


max depth:  5 , num parallel tree:  2 gamma:  87
------------------------------------------------------------
max depth:  5 , num parallel tree:  2 gamma:  88
------------------------------------------------------------
max depth:  5 , num parallel tree:  2 gamma:  90
------------------------------------------------------------
max depth:  5 , num parallel tree:  2 gamma:  91
------------------------------------------------------------
max depth:  5 , num parallel tree:  4 gamma:  87
------------------------------------------------------------
max depth:  5 , num parallel tree:  4 gamma:  88
------------------------------------------------------------
max depth:  5 , num parallel tree:  4 gamma:  90
------------------------------------------------------------
max depth:  5 , num parallel tree:  4 gamma:  91
------------------------------------------------------------
max depth:  5 , num parallel tree:  5 gamma:  87
------------------------------------------------------------
max depth:

max depth:  5 , num parallel tree:  2 gamma:  91
------------------------------------------------------------
max depth:  5 , num parallel tree:  4 gamma:  87
------------------------------------------------------------
max depth:  5 , num parallel tree:  4 gamma:  88
------------------------------------------------------------
max depth:  5 , num parallel tree:  4 gamma:  90
------------------------------------------------------------
max depth:  5 , num parallel tree:  4 gamma:  91
------------------------------------------------------------
max depth:  5 , num parallel tree:  5 gamma:  87
------------------------------------------------------------
max depth:  5 , num parallel tree:  5 gamma:  88
------------------------------------------------------------
max depth:  5 , num parallel tree:  5 gamma:  90
------------------------------------------------------------
max depth:  5 , num parallel tree:  5 gamma:  91
------------------------------------------------------------
max depth:

In [151]:
# pd.DataFrame(my_dict).to_excel('output_v1.4.xlsx', index = False)

In [ ]:

# X_validation_prob = list(xgb_cl.predict_proba(X_validation)[:,1])


# validation['Prob_score_XGB'] = X_validation_prob


# print("ROC AUC Score for validation:", roc_auc_score(y_validation, X_validation_prob) * 100)


In [ ]:
feat_imp = pd.DataFrame.from_dict(zip(list(X_train.columns), xgb_cl.feature_importances_*100)).sort_values(by = 1, ascending = False)
# feat_imp.to_excel('feature_imp_xgb.xlsx', index = False)
feat_imp

In [ ]:
feat_imp[0].values

In [ ]:
# print()
print("ROC AUC Score for train:", roc_auc_score(y_train, y_pred_xg1) * 100)
print("ROC AUC Score for test:", roc_auc_score(y_test, y_pred_xg) * 100)
# print("ROC AUC Score for valid:", roc_auc_score(y_validation, y_pred_xg_v) * 100)

In [ ]:
# xg_train_ks.to_csv('xgboost_ks_train.csv', index = False)
xg_train_ks.reset_index()['cum_eventrate'][2]

In [ ]:
# xg_test_ks.to_csv('xgboost_ks_test.csv', index = False)
xg_test_ks

In [ ]:
# xg_validation_ks.to_csv('xgboost_ks_validation.csv', index = False)
# xg_validation_ks